<a href="https://colab.research.google.com/github/hellomikelo/hackathon-cohere-qdrant/blob/dev-prototype/pubmed_qa_cohere_qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cohere Qdrant hackathon 

In [ ]:
!pip install datasets qdrant_client=="0.11.0" cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
COHERE_API_KEY = "XXXX"
QDRANT_API_KEY = "XXXX"

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("pubmed_qa", "pqa_labeled")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import cohere

In [ ]:
cohere_client = cohere.Client(COHERE_API_KEY)
cohere_client

In [ ]:
embeddings = cohere_client.embed(
    texts=["A test sentence"],
    model="large",
)
vector_size = len(embeddings.embeddings[0])
vector_size

4096

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client import models
from qdrant_client.http import models as rest

In [ ]:
qdrant_client = QdrantClient(
    host="xyz-example.eu-central.aws.staging-cloud.qdrant.io", 
    prefer_grpc=True,
    api_key=QDRANT_API_KEY,
)

In [ ]:
qdrant_client.recreate_collection(
    collection_name="pubmed_qa",
    vectors_config=models.VectorParams(
        size=vector_size, 
        distance=rest.Distance.COSINE
    ),
)

In [ ]:
answer_response = cohere_client.embed(
    texts=dataset["train"]["long_answer"],
    model="large",
)
vectors = [list(map(float, vector)) for vector in answer_response.embeddings]
ids = [entry["pubid"] for entry in dataset["train"]]

In [ ]:
qdrant_client.upsert(
    collection_name="pubmed_qa", 
    points=rest.Batch(
        ids=ids,
        vectors=vectors,
        payloads=list(dataset["train"]),
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
question_response = cohere_client.embed(
    texts=dataset["train"]["question"],
    model="large",
)

In [ ]:
from tqdm import tqdm

In [ ]:
k_max = 10
answer_positions = []
for embedding, pubid in tqdm(zip(question_response.embeddings, ids)):
    response = qdrant_client.search(
        collection_name="pubmed_qa",
        query_vector=embedding,
        limit=k_max,
    )

    answer_ids = [record.id for record in response]
    if pubid in answer_ids:
        answer_positions.append(answer_ids.index(pubid))
    else:
        answer_positions.append(-1)


for k in range(1, k_max + 1):
    correct_answers = len(
        list(
            filter(lambda x: 0 <= x < k, answer_positions)
        )
    )
    print(f"accuracy@{k} =", correct_answers / len(dataset["train"]))

1000it [00:19, 50.53it/s]

accuracy@1 = 0.877
accuracy@2 = 0.921
accuracy@3 = 0.942
accuracy@4 = 0.95
accuracy@5 = 0.956
accuracy@6 = 0.96
accuracy@7 = 0.964
accuracy@8 = 0.971
accuracy@9 = 0.976
accuracy@10 = 0.977
